In [ ]:
import requests
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
import pandas as pd
import json
import numpy as np
import dask.dataframe as dd


# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")

today = date.today()

# 設定
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 100)
# NumPy配列の表示オプションを設定
np.set_printoptions(threshold=np.inf)
# 配列の表示形式を科学表記ではなく通常の浮動小数点数に設定
np.set_printoptions(suppress=True)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
# plt.rcParams['font.family'] = 'Meiryo'

In [ ]:
# CSVファイルを読み込む
file_path = 'data/input/user_info_cleansing.csv'  # ファイルパスを適切に設定してください
df1 = dd.read_csv(file_path).drop(['Unnamed: 0'], axis=1).compute()
file_path = 'data/input/gacha_history.csv'  # ファイルパスを適切に設定してください
df2 = dd.read_csv(file_path).compute()
file_path = 'data/input/point_history_cleansing.csv'  # ファイルパスを適切に設定してください
# column_types = {
#     'id' : np.float16,
#     'user_id' : int,
#     'series_id' : np.float16,
#     'shop_id' : np.float16,
#     'shop_name' : str,
#     'card_id' : str,
#     'リサイクル分類ID' : np.float16,
#     'amount' : np.float16,
#     'amount_kg' : np.float16,
#     'point' : np.float16,
#     'total_point' : np.float16,
#     'status' : np.float16,
#     'total_amount' : np.float16,
#     'coin' : np.float16,
#     'rank_id' : np.float16,
#     'use_date' :   'datetime64[ns]',
#     'created_at' : 'datetime64[ns]',
#     'updated_at' : 'datetime64[ns]',
#     '支店ID' : np.float16,
#     'super' : str,
#     'prefectures' : str,
#     'municipality' : str,
#     'shop_name_1' :  str,
#     'shop_id_1' :    str,
#     'created_at_1' : 'datetime64[ns]',
#     'updated_at_1' : 'datetime64[ns]',
#     'store_latitude' : np.double,
#     'store_longitude' : np.double,
# }
df3 = dd.read_csv(
    file_path,
    dtype={
        'series_id': 'Int64',
        'shop_id': 'Int64',
        'shop_id_1': str,
        'リサイクル分類ID': 'Int64',
        '支店ID': 'Int64',
        'rank_id': 'Int64'
    }
).drop(['Unnamed: 0'], axis=1).compute()

file_path = 'data/input/ユーザー基本情報_2023-12-21.csv'
df4 = dd.read_csv(file_path, encoding='sjis').compute()

# 'カード登録日' 列を datetime オブジェクトに変換（日付の形式は適宜調整してください）
df4['カード登録日'] = pd.to_datetime(df4['カード登録日'], format='%Y/%m/%d')
# サービス利用開始からの経過日数
df4['サービス利用開始からの経過日数'] = (pd.to_datetime(today) - df4['カード登録日']).dt.days
# サービス利用開始からの経過月数
df4['サービス利用開始からの経過月数'] = (pd.to_datetime(today) - df4['カード登録日']).dt.days / 30
# 経過月数が1未満は1に補正する。月数で割ると月平均が増えてしまうため
df4['サービス利用開始からの経過月数'] = df4['サービス利用開始からの経過月数'].apply(lambda x: 1 if x<1 else x)

# 不正データnan変換
df1 = df1.replace('N', np.nan)
df1 = df1.replace('nan', np.nan)
df2 = df2.replace('N', np.nan)
df2 = df2.replace('nan', np.nan)
df3 = df3.replace('N', np.nan)
df3 = df3.replace('nan', np.nan)
df4 = df4.replace('N', np.nan)
df4 = df4.replace('nan', np.nan)

#pointhistoryの事務局操作データは除外
df3 = df3[df3['status'] != 3]

結合

In [ ]:
df_user_merge = pd.merge(df1, df4, left_on='id', right_on='利用者ID', how='inner')
# 毎月平均リサイクル量
df_user_merge['毎月平均リサイクル量'] = df_user_merge['total_recycle_amount'] / df_user_merge['サービス利用開始からの経過月数']
# display(df_user_merge[['total_recycle_amount','サービス利用開始からの経過日数','毎月平均リサイクル量']])

df_merge_gacha = pd.merge(df2, df_user_merge, left_on='user_uid', right_on='id', how='left')
df_merge_gacha = df_merge_gacha.drop(['id_x','id_y'], axis=1) 

df_merge_point = pd.merge(df3, df_user_merge, left_on='user_id', right_on='id', how='left')
df_merge_point = df_merge_point.drop(['id_x','id_y'], axis=1) 
df_merge_point = df_merge_point[~df_merge_point['サービス利用開始からの経過日数'].isna()]
df_merge_point = df_merge_point[~df_merge_point['サービス利用開始からの経過月数'].isna()]


In [ ]:
# print(np.sort(df_merge_point['サービス利用開始からの経過月数'].unique()))

#### 特徴量候補を列挙
ユーザ毎
- 毎月平均リサイクル量
- 毎月平均リサイクル回数
- 毎月平均クラブコインの使用量
- 毎月平均ガチャの取得量
- 毎月平均ガチャの使用量
- 平均rank
- 店舗との距離←緯度経度をgeocodeで算出
- 店舗のリサイクル許容量
- 性別
- 年代
- カード種類
- サービス利用開始からの経過日数 ←特徴量としてはいらないかも

### 特徴量算出

#### - 毎月平均リサイクル回数

In [ ]:
# ユーザー別リサイクル回数
count = df_merge_point.groupby('user_id').size()
count = count.to_frame('リサイクル回数')
count = count.reset_index()

df_user_merge = pd.merge(df_user_merge, count, left_on='id', right_on='user_id', how='inner')
df_user_merge['月平均リサイクル回数'] = df_user_merge['リサイクル回数'] / df_user_merge['サービス利用開始からの経過月数']

#### - 毎月平均クラブコインの使用量

In [ ]:
used_coin = df_merge_point[df_merge_point['coin']<0].groupby('user_id')['coin'].sum()
used_coin = used_coin.to_frame('消費クラブコイン合計量').reset_index()

df_merge_point = pd.merge(df_merge_point, used_coin, on='user_id',how='left')
df_merge_point['消費クラブコイン合計量'] = df_merge_point['消費クラブコイン合計量'].fillna(0)
df_merge_point['平均クラブコインの使用量'] = df_merge_point['消費クラブコイン合計量'] / df_merge_point['サービス利用開始からの経過月数']

In [ ]:
df_merge_point

#### ワンホットエンコーディング
#### 正規化